In [2]:
!nvidia-smi

Mon Dec 14 19:49:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64       Driver Version: 440.64       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:08:00.0 Off |                    0 |
| N/A   33C    P0    28W / 250W |     10MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:0D:00.0 Off |                    0 |
| N/A   37C    P0    30W / 250W |      0MiB / 16280MiB |      4%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [16]:
import torch
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
import torchvision.datasets
from torchvision import transforms

In [3]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [4]:
import torchvision.models as models
#model = models.vgg16(pretrained=False)

In [5]:
PATH = 'CIFAR10_dense161.pth'
#PATH = 'CIFAR10_ResNet18.pth'
model = torch.load(PATH)
model.eval()

DataParallel(
  (module): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(144, eps=1e-05, momentu

In [7]:
!pip install -U tensorly

     |████████████████████████████████| 106 kB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 154 kB 4.6 MB/s eta 0:00:01
  Created wheel for tensorly: filename=tensorly-0.5.1-py3-none-any.whl size=152072 sha256=bb7d4476188c0c755549a9690421594518e6eb348361347de8ebecdff038be08
  Stored in directory: /root/.cache/pip/wheels/64/1e/b0/54c0906c4305decc4466e1296e2456a6b02c2cdb1a5a69141e
Successfully built tensorly
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [8]:
from decompositions import cp_decomposition_conv_layer, tucker_decomposition_conv_layer

In [9]:
import tensorly as tl
import tensorly

In [15]:
for params in model.parameters():
    print(params)

Parameter containing:
tensor([[[[-1.8333e-02, -9.8090e-03, -4.8354e-03,  ...,  4.2858e-02,
            2.0895e-02,  2.1024e-02],
          [ 6.0225e-03,  1.7785e-03, -8.2574e-02,  ..., -2.5827e-01,
           -9.6055e-02,  3.9424e-02],
          [-1.5927e-02,  5.8963e-02,  3.0458e-01,  ...,  5.3739e-01,
            2.6658e-01,  5.5593e-02],
          ...,
          [-2.5203e-02,  2.0810e-02,  8.4297e-02,  ..., -3.5054e-01,
           -4.2998e-01, -2.6248e-01],
          [ 1.1157e-02,  1.4853e-02,  8.1510e-02,  ...,  4.2544e-01,
            4.3158e-01,  1.9987e-01],
          [-3.6949e-02, -2.4581e-02, -2.4902e-02,  ..., -1.9364e-01,
           -1.0342e-01, -6.1984e-04]],

         [[-2.9159e-02, -3.7152e-02, -4.2008e-02,  ...,  4.4127e-03,
           -2.1987e-03, -1.1369e-03],
          [ 4.1130e-02,  1.9261e-02, -7.8652e-02,  ..., -2.9507e-01,
           -1.1755e-01,  4.0709e-02],
          [-1.2765e-02,  9.4427e-02,  4.0496e-01,  ...,  7.2405e-01,
            3.8860e-01,  1.2189e-01]

In [13]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [11]:
model = model.module

In [38]:
for name, module in model.named_modules():
    if type(module) not in (nn.Sequential, 
                            torchvision.models.densenet._DenseLayer,
                            torchvision.models.densenet.DenseNet,
                            torchvision.models.densenet._DenseBlock,
                           torchvision.models.densenet._Transition):
        print(name, module) 

features.conv0 Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
features.norm0 BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
features.relu0 ReLU(inplace=True)
features.pool0 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
features.denseblock1.denselayer1.norm1 BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
features.denseblock1.denselayer1.relu1 ReLU(inplace=True)
features.denseblock1.denselayer1.conv1 Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
features.denseblock1.denselayer1.norm2 BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
features.denseblock1.denselayer1.relu2 ReLU(inplace=True)
features.denseblock1.denselayer1.conv2 Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
features.denseblock1.denselayer2.norm1 BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [44]:
named_modules = [(name,module) for module in model.named_modules() if type(module) not in (nn.Sequential, 
                                                torchvision.models.densenet._DenseLayer,
                                                torchvision.models.densenet.DenseNet,
                                                torchvision.models.densenet._DenseBlock,
                                                torchvision.models.densenet._Transition)                
                            ]
len(named_modules)

573

In [45]:
getattr(model, name)

#setattr(object,name,value)

Linear(in_features=2208, out_features=10, bias=True)

In [46]:
model.eval()
model.cpu()
tl.set_backend('pytorch')
N = len(named_modules)
for i, (name, module) in enumerate(named_modules):
    #??????????? first conv layer
    if i >= N - 2:
        break
    if isinstance(module, torch.nn.modules.conv.Conv2d):
        conv_layer = module
        if True:
            rank = max(conv_layer.weight.data.numpy().shape)//3
            decomposed = cp_decomposition_conv_layer(conv_layer, rank)
        else:
            decomposed = tucker_decomposition_conv_layer(conv_layer)
        
        setattr(model, name ,decomposed)

torch.save(model, 'CIFAR10_dense161_compressed_v1.pth')